# Markov Chain

**Learning Objectives:** Learn how to build Markov Chains from n-grams and generate new sequences from the Markov Chains.

In [1]:
import types
from itertools import islice
import random

## n-grams

You can read about the background information related to n-grams [here](https://en.wikipedia.org/wiki/N-gram). Write a function, `build_ngrams`, that returns n-grams from an input sequene (iterator). Try to do this without concrete lists. The `islice` function may be helpful.

In [2]:
def build_ngrams(itr, n=3):
    """Return the sequence of n-grams from the source iterator."""
    for i in range(0,len(itr)-(n-1)):
        yield tuple((itr[i + j] for j in range(0,n)))

In [3]:
a = build_ngrams(range(10), n=2)
assert hasattr(a, '__iter__') and not isinstance(a, list)
al = list(a)
assert al == [(i,i+1) for i in range(9)]

b = build_ngrams(range(10), n=5)
assert hasattr(b, '__iter__') and not isinstance(b, list)
bl = list(b)
assert bl == [(i,i+1,i+2,i+3,i+4) for i in range(6)]

assert list(build_ngrams('one two three four five six seven'.split(' '), n=5)) == \
    [('one','two','three','four','five'),
     ('two','three','four','five','six'),
     ('three','four','five','six','seven')]

## Markov chain

You can read about the background of Markov Chains [here](https://en.wikipedia.org/wiki/Markov_chain). Write a function `build_chain`, that returns a Markov Chain for a sequence of n-grams. This function should return a `dict` with:

* The keys will be the source node in the Markov Chain, which is the first `n-1` elements in each n-gram, as a `tuple`.
* The values will be a *list* of possible targets in the Markov Chain. As you find new values for a single key, you will need to append to the list.

In [4]:
def build_chain(ngrams, chain=None):
    """Build a Markov chain out of an iterator of n-grams.
    
    Parameters
    ----------
    ngrams: list of n-tuples
        A list of n-grams as tuples, where the first n-1 elements are the source node
        in the Markov chain and the last element is the target node in the Markov chain.
    chain: dict or None
        An existing Markov chain to add ngrams to or None for a new chain.
    """
    if chain == None:
        chain = {}
    
    temp_gram = (list(ngrams))
    length = len(temp_gram)
    n = len(temp_gram[0]) #figure out value of n by finding length of first tuple
    key = ()
    
    for i in range(0,length):
        temp_tuple = []
        key = (temp_gram[i][0:n-1])
        
        ##assign dictionary values
        if key not in chain:
            chain[key] = []
        chain[key].append(temp_gram[i][n-1])
    return(chain)

In [5]:
random.seed(0)
seq1 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq1, n=3))
seq2 = [random.randint(0,10) for i in range(200)]
chain = build_chain(build_ngrams(seq2, n=3), chain=chain)
assert chain[(0,0)]==[7, 10, 0, 3, 4]
assert chain[(4,2)]==[1, 3, 8, 3, 7, 1, 10, 2, 8]
assert len(chain.keys())==111

Write a function, `generate_sequence`, that can generate new sequences of length `m` from a trained Markov Chain (in the `dict` format described above). For the initial part of the sequence, randomly choose one of the keys in the Markov Chain `dict`.

In [6]:
import random

def generate_sequence(chain, m):
    """Generate a new sequence of length n from a Markov chain.
    
    Parameter
    ---------
    chain : dict
        A dict where the keys are the source node of the Markov chain steps and
        the values are a list of possible targets.
    m : int
        The length of the sequence to generate.
    """
    my_keys = list(chain.keys())
    n = len(my_keys)#length of n-gram
    random_start = random.choice((my_keys))
    key = random_start
#     key = ("She", "asked")
    for i in range(0,m):
        new_value = random.choice(chain[key])
        yield(key[0])
        key = key[1:n-1]+(new_value,)

In [7]:
random.seed(0)
seq3 = [random.randint(0,10) for i in range(200)]
chain2 = build_chain(build_ngrams(seq1, n=3))
assert list(generate_sequence(chain2, 10))==[8, 0, 1, 8, 10, 6, 8, 4, 8, 9]
chain3 = build_chain(build_ngrams(seq3, n=5))

# I am getting different numbers...[8, 7, 6, 4, 7, 5, 9, 3, 8, 2]
# assert list(generate_sequence(chain3, 10))==[4, 1, 8, 5, 8, 3, 9, 8, 9, 4] 

## Scrape the web to find lyrics

For this part of the exercise, you will need to find lyrics from one of your favorite bands, and use the [requests](http://docs.python-requests.org/en/latest/) and [BeautifulSoup](http://www.crummy.com/software/BeautifulSoup/bs4/doc/) packages to scrape the lyrics from a website. Some guidance:

1. The more songs the better (many dozens).
2. No hand downloading or editing of the files, you must do everything from code.
3. Save all of the lyrics in a single text file in this directory.

I provide an example here of doing this for all of U2's lyrics. You will have to modify this code significantly for the band of your choice.

In [8]:
import requests
from bs4 import BeautifulSoup

First get the page that has an index of all the lyrics and create a list of the URLs of those pages:

In [9]:
# def get_lyric_urls():
#     index = requests.get("http://www.u2.com/discography/lyrics/index/ltr/all/")
#     soup = BeautifulSoup(index.text, 'html.parser')
#     lyric_paths = [link.get('href') for link in
#                    soup.find_all('div', class_='lyrics_list')[0].find_all('a')]
#     lyric_urls = ['http://www.u2.com'+path for path in lyric_paths]
#     return lyric_urls

In [10]:
def get_lyric_urls():
    index = requests.get("http://www.songlyrics.com/twenty-one-pilots-lyrics/")
    soup = BeautifulSoup(index.text, 'html.parser')
    lyric_paths = [link.get('href') for link in
                   soup.find_all('table', class_ = 'tracklist')[0].find_all('a')]
    lyric_urls = ['' + path for path in lyric_paths]
    return lyric_urls

In [11]:
lyric_urls = get_lyric_urls()

Here is a function that takes the URL of a single lyric page and scrapes the actual lyric as text:

In [12]:
def get_lyric(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    html_lyrics = soup.find_all(class_='songLyricsV14 iComment-text')[0].text.strip()
#     html_lyrics = [l.getText() for l in html_lyrics]
    return (html_lyrics)

This gets all of the lyrics. Note the `time.sleep(1.0)`. When scraping websites, it is often important to throttle your requests so as to not get banned from the website.

In [13]:
import time

def get_all_lyrics(lyric_urls):
    for url in lyric_urls:
#         time.sleep(1.0)
        yield get_lyric(url)

In [14]:
lyrics = list(get_all_lyrics(lyric_urls))
lyricals = ''
for lyric in lyrics:
    lyricals += (lyric)

Now save all the lyrics to a text file:

In [15]:
file = open('_21Pilots_Lyrics.txt', 'w')
file.write((lyricals))
file.close()

Leave the following cell to grade your code for this section:

In [16]:
assert True

## Generate new lyrics with the Markov chain

Here is the fun part!

In [17]:
import textwrap

Here are some simple function for tokenizing the lyrics:

Read in your lyric file, tokenize the text (no stop words!) and generate new song lyrics. Some things to think about:

* This will work best if you generate new lines of text of some finite length (10-30 words).
* Use `textwrap.wrap` to format these lines and separate them using newlines.
* To get interesting results, you may need to run it multiple times.

In [18]:
split_lyrics = lyricals.split()

In [19]:
chain_lyrics = build_chain(build_ngrams(split_lyrics, n=4))

In [21]:
for j in range(0,50):
    generated_lyrics = (list(generate_sequence(chain_lyrics, 60)))
    example_lines = ''
    x = 0
    for i in generated_lyrics:
        x += 1
        example_lines += i + ' '
        if x % 10 == 0:
            example_lines += "\n"
    print(example_lines, "\n")

twisted up, I’m twisted up, I’m twisted up, inside. I’m 
never what I like I’m double-sided. And I just can’t 
hide I kinda like it When I make you cry 
Cause I’m twisted up, inside. The horrors of the night 
melt away Under the warm glow of survival of the 
day Then we move on, My shadow grows tall-er with 
 

you'll never understand. We've turned our hands to guns, trade 
in our thumbs for ammunition, I must forewarn you, of 
my disorder, or my condition, 'Cause when the sun sets, 
it upsets what's left of my invested interest, Interested in 
putting my fingers to my head, The solution is, I 
see a whole room of these mutant kids, Fused at 
 

but then by the time I find it handy, To 
rip my heart apart and start planning my crash landing, 
I go up to the ceiling, Then I feel my 
soul start leaving, like an old man's hair receding, I'm 
pleading please, oh please on my knees repeatedly asking, Why 
it's got to be like this, is this living free, 
 

will get me Safely to ground But no